## UTILIZANDO **GRADIO** PARA VALIDAÇÃO DE DETECÇÃO DE OBJETOS

- **Gradio** é uma ferramenta em Python que simplifica a criação e compartilhamento de demos ou aplicativos web para modelos de ML, APIs ou funções, sem necessidade de experiência em JavaScript, CSS ou hospedagem web.
- Neste _notebook_, abordaremos a criação de uma **interface gráfica** básica utilizando o **Gradio** e como executar um modelo de **detecção de objetos** em conjunto com essa interface.

#### 🔎 Verificar se está sendo utilizado CPU ou GPU

In [1]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

#
---

### 1º PASSO: IMPORTAR AS BIBLIOTECAS

In [2]:
import os
import zipfile

import gradio as gr
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont

c:\Users\saothalm\AppData\Local\miniconda3\envs\vertex_course\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 
---

### 2º PASSO: CRIAR A INTERFACE RESPONSÁVEL POR EXECUTAR O MODELO

Vamos empregar **variáveis globais** para simplificar o código, porém essa não é a única forma de implementar a função. 
_(Sinta-se à vontade para otimizar o código se desejar)_

In [3]:
# Diretório onde o modelo está armazenado
DATA_PATH = os.path.join(os.pardir, "data", "models", "detec4")

# Nome do modelo .tflite a ser utilizado
MODEL_FILE = "t5_battery_detec.tflite"

# Caminho completo do diretório até o arquivo do modelo
MODEL_PATH = os.path.join(DATA_PATH, MODEL_FILE)

#
---

### 3º PASSO: EXTRAIR AS CATEGORIAS DO MODELO QUE SERÁ EXECUTADO

O código abaixo imprime no _output_ quais categorias existem dentro do modelo carregado na variável global **MODEL_PATH**

In [4]:
with zipfile.ZipFile(MODEL_PATH, "r") as archive:
    for file in archive.namelist():
        LABELS = archive.read(file).decode(encoding="utf-8").split()
        display(LABELS)

['flex', 'cr2', 'eve']

In [5]:
LABELS = LABELS[0:]
display(f"N° de Classes: {len(LABELS)}", LABELS)

'N° de Classes: 3'

['flex', 'cr2', 'eve']

O código abaixo extrai o arquivo **dict.txt** do modelo e o salva no mesmo diretório

In [6]:
with zipfile.ZipFile(MODEL_PATH, "r") as zip_ref:
    zip_ref.extractall(DATA_PATH)

#
---

### 4º PASSO: CRIAR O INTERPRETADOR QUE USARÁ O MODELO E ALOCAR NA MEMÓRIA

In [7]:
# Criação do interpretador do modelo
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)

# Alocação na memória
interpreter.allocate_tensors()

#
---

### 5º PASSO: CRIAR FUNÇÃO DE PREVISÃO COM O INTERPRETADOR DO MODELO

Parâmetros que serão utilizados pelo _*predict*_:
- Cria uma lista de cores que serão utilizadas para *visualização*, utilizando uma _seed_ para que não altere o tempo todo.

In [8]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype=np.uint8)

In [9]:
dict(zip(LABELS, COLORS))

{'flex': array([102, 220, 225], dtype=uint8),
 'cr2': array([ 95, 179,  61], dtype=uint8),
 'eve': array([234, 203,  92], dtype=uint8)}

- Carrega o modelo .tflite:

In [10]:
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

- Esta é a função que irá pré-processar a imagem de entrada para alimentar o modelo TFLite:

In [11]:
def preprocess_image(interpreter, image):
    input_details = interpreter.get_input_details()[0]
    model_height, model_width = input_details["shape"][1:3]

    # Redimensionar a imagem
    image_resized = tf.image.resize(image, (model_height, model_width))

    # Normalizar a imagem
    img_array = tf.cast(image_resized, tf.float32) / 255.0

    # Verificar se é necessário escalar e quantizar
    if input_details["dtype"] == tf.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        img_array = img_array / input_scale + input_zero_point
        img_array = tf.cast(img_array, input_details["dtype"])

    # Adicionar dimensão de lote, se necessário
    img_array = tf.expand_dims(img_array, axis=0)

    return img_array

- Esta é a função retornará uma lista de resultados de detecção. Cada resultado em um dicionário com infos sobre o objeto.

In [12]:
def detect_objects(interpreter, image_array, threshold):

    interpreter.set_tensor(interpreter.get_input_details()[0]["index"], image_array)
    interpreter.invoke()

    boxes = np.squeeze(
        interpreter.get_tensor(interpreter.get_output_details()[0]["index"])
    )
    classes = np.squeeze(
        interpreter.get_tensor(interpreter.get_output_details()[1]["index"])
    )
    scores = np.squeeze(
        interpreter.get_tensor(interpreter.get_output_details()[2]["index"])
    )
    count = int(
        np.squeeze(interpreter.get_tensor(interpreter.get_output_details()[3]["index"]))
    )
    results = []
    for i in range(count):
        if scores[i] >= threshold:
            result = {
                "bounding_box": boxes[i],
                "class_id": classes[i],
                "score": scores[i],
            }
            results.append(result)
    return results

- Esta é a função final de detecção de objetos em uma imagem de entrada, que desenha os resultados da detecção.

In [13]:
def predict(image, threshold):
    global LABELS
    global interpreter

    new_image = image.copy()

    # Load the input image and preprocess it
    preprocessed_image = preprocess_image(interpreter, new_image)

    # Run object detection on the input image
    results = detect_objects(interpreter, preprocessed_image, threshold=threshold)
    display(results)

    # Create a Pillow ImageDraw object
    draw = ImageDraw.Draw(new_image)

    for obj in results:
        # Convert the object bounding box from relative coordinates to absolute
        # coordinates based on the original image resolution
        ymin, xmin, ymax, xmax = obj["bounding_box"]

        xmin = int(xmin * new_image.width)
        xmax = int(xmax * new_image.width)
        ymin = int(ymin * new_image.height)
        ymax = int(ymax * new_image.height)

        # Find the class index of the current object
        class_id = int(obj["class_id"])

        # Draw the bounding box and label on the image
        color = [int(c) for c in COLORS[class_id]]
        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline=tuple(color), width=2)

        # Make adjustments to make the label visible for all objects
        # font = ImageFont.load_default()
        font = ImageFont.truetype("arial.ttf", 40)
        label = f"{LABELS[class_id]} {round(obj['score'] * 100, 2)}%"

        text_position = (xmin, ymin)

        text_left, text_top, text_right, text_bottom = draw.textbbox(
            text_position, label, font=font
        )
        draw.rectangle(
            (text_left - 5, text_top - 5, text_right + 5, text_bottom + 5),
            fill=tuple(color),
        )

        draw.text(text_position, label, fill="black", font=font)

    return new_image

# 
---

### 6º PASSO: CRIAR A INTERFACE COM O GRADIO

Criação da interface que utiliza da função **_predict(image)_**

Esta interface possui um configurador de _*threshold*_.

In [14]:
gr.Interface(
    fn=predict,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(
            0.1,
            1.0,
            step=0.05,
            value=0.5,
            label="Threshold",
            info="Choose a confident percentage value",
        ),
    ],
    outputs=[gr.Image()],
).launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


[]

[]

[{'bounding_box': array([0.36187315, 0.22808257, 0.5147912 , 0.42725036], dtype=float32),
  'class_id': np.float32(2.0),
  'score': np.float32(0.81640625)}]

####
---

#### 💡VALIDE O MODELO ATRAVÉS DA INTERFACE
- É possível testar o modelo pelo próprio _output_ da célula do notebook
- Outra opção é clicar no link gerado e testá-lo em seu _browser_ de preferência